In [5]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

%matplotlib inline 

In [17]:
spark = SparkSession.builder.getOrCreate()
df = spark.read.options(header=True).csv("epl_results_2022-23.csv")
df.show()

+---+----------+-----+--------------+--------------+----+----+---+----+----+---+------------+---+---+---+---+---+---+---+---+---+---+---+---+
|Div|      Date| Time|      HomeTeam|      AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR|     Referee| HS| AS|HST|AST| HF| AF| HC| AC| HY| AY| HR| AR|
+---+----------+-----+--------------+--------------+----+----+---+----+----+---+------------+---+---+---+---+---+---+---+---+---+---+---+---+
| E0|05/08/2022|20:00|Crystal Palace|       Arsenal|   0|   2|  A|   0|   1|  A|    A Taylor| 10| 10|  2|  2| 16| 11|  3|  5|  1|  2|  0|  0|
| E0|06/08/2022|12:30|        Fulham|     Liverpool|   2|   2|  D|   1|   0|  H|    A Madley|  9| 11|  3|  4|  7|  9|  4|  4|  2|  0|  0|  0|
| E0|06/08/2022|15:00|   Bournemouth|   Aston Villa|   2|   0|  H|   1|   0|  H|    P Bankes|  7| 15|  3|  2| 18| 16|  5|  5|  3|  3|  0|  0|
| E0|06/08/2022|15:00|         Leeds|        Wolves|   2|   1|  H|   1|   1|  D|     R Jones| 12| 15|  4|  6| 13|  9|  6|  4|  2|  0|  0|  0|
| E0|0

In [22]:
# partition by Home team
df_home = df.repartition("HomeTeam")

In [23]:
df_home.show()
df_home.count()

+---+----------+-----+---------+-----------+----+----+---+----+----+---+------------+---+---+---+---+---+---+---+---+---+---+---+---+
|Div|      Date| Time| HomeTeam|   AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR|     Referee| HS| AS|HST|AST| HF| AF| HC| AC| HY| AY| HR| AR|
+---+----------+-----+---------+-----------+----+----+---+----+----+---+------------+---+---+---+---+---+---+---+---+---+---+---+---+
| E0|06/08/2022|15:00|Tottenham|Southampton|   4|   1|  H|   2|   1|  H|  A Marriner| 18| 10|  8|  2| 11|  6| 10|  2|  3|  0|  0|  0|
| E0|20/08/2022|12:30|Tottenham|     Wolves|   1|   0|  H|   0|   0|  D|    S Hooper| 11| 20|  4|  3| 10|  8|  8|  6|  2|  2|  0|  0|
| E0|03/09/2022|15:00|Tottenham|     Fulham|   2|   1|  H|   1|   0|  H|   S Attwell| 23|  9| 10|  3|  9|  6| 10|  3|  3|  4|  0|  0|
| E0|17/09/2022|17:30|Tottenham|  Leicester|   6|   2|  H|   2|   2|  D|    S Hooper| 16| 19| 11|  7|  9|  9|  6|  1|  1|  2|  0|  0|
| E0|15/10/2022|17:30|Tottenham|    Everton|   2|   0|  H|   0

337

In [24]:
df_home.createOrReplaceTempView("EPL")

In [45]:
tt_df = spark.sql("""
WITH new_table AS (
    SELECT *, CASE WHEN FTR = "H" THEN 1 WHEN FTR = "A" THEN 2 ELSE 3 END AS FTR_NUMERIC
    FROM EPL
  )
  SELECT *
  FROM new_table
  WHERE HomeTeam = "Tottenham"
""")
tt_df.show()

+---+----------+-----+---------+-----------+----+----+---+----+----+---+-----------+---+---+---+---+---+---+---+---+---+---+---+---+-----------+
|Div|      Date| Time| HomeTeam|   AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR|    Referee| HS| AS|HST|AST| HF| AF| HC| AC| HY| AY| HR| AR|FTR_NUMERIC|
+---+----------+-----+---------+-----------+----+----+---+----+----+---+-----------+---+---+---+---+---+---+---+---+---+---+---+---+-----------+
| E0|06/08/2022|15:00|Tottenham|Southampton|   4|   1|  H|   2|   1|  H| A Marriner| 18| 10|  8|  2| 11|  6| 10|  2|  3|  0|  0|  0|          1|
| E0|20/08/2022|12:30|Tottenham|     Wolves|   1|   0|  H|   0|   0|  D|   S Hooper| 11| 20|  4|  3| 10|  8|  8|  6|  2|  2|  0|  0|          1|
| E0|03/09/2022|15:00|Tottenham|     Fulham|   2|   1|  H|   1|   0|  H|  S Attwell| 23|  9| 10|  3|  9|  6| 10|  3|  3|  4|  0|  0|          1|
| E0|17/09/2022|17:30|Tottenham|  Leicester|   6|   2|  H|   2|   2|  D|   S Hooper| 16| 19| 11|  7|  9|  9|  6|  1|  1|  2|  0|  

In [39]:

# convert to pandas dataframe
tt_df_pandas = tt_df.toPandas()

X = tt_df_pandas[['HTHG', 'HTAG', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR']]
y = tt_df_pandas['FTR']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)
lm = LinearRegression()

In [43]:
lm.fit(X_train,y_train)

# predictions = lm.predict(X_test)

# plt.scatter(y_test,predictions)

ValueError: could not convert string to float: 'A'